## CRF What's Wrong with Participants?

In [ ]:
from crf import *
from crf_support import get_all_data, compare_tags, filter_phrase

import os, time

from features_generator import abstracts2features, get_genia_tags, sanity_check

from gensim.models import Word2Vec

In [ ]:
tag = 'P'
eval_tags = [tag]

Get data

In [ ]:
# Get train data
train_tokens, train_tags = get_all_data('train', tag)
train_genia_tags = get_genia_tags('train')

In [ ]:
# Get dev data
dev_tokens, dev_tags = get_all_data('dev', tag)
dev_genia_tags = get_genia_tags('dev')

In [ ]:
# Get test data
test_tokens, test_tags = get_all_data('test', tag)
test_genia_tags = get_genia_tags('test')

Compute features

In [ ]:
pubmed_w2v_name = 'PubMed-w2v.bin'
pubmed_w2v = Word2Vec.load_word2vec_format(pubmed_w2v_name, binary=True)

In [ ]:
pubmed_wiki_w2v_name = 'wikipedia-pubmed-and-PMC-w2v.bin'
pubmed_wiki_w2v = Word2Vec.load_word2vec_format(pubmed_wiki_w2v_name, binary=True)

In [ ]:
# Set options
big_options_string = 'left_neighbors=3 right_neighbors=3 inside_paren pos chunk iob named_entity \
inside_paren_neighbors pos_neighbors chunk_neighbors iob_neighbors named_entity_neighbors \
chunk_end chunk_end_neighbors same_chunk_neighbors \
one_hot one_hot_neighbors w2v_model=pubmed w2v w2v_neighbors w2v_size=10 cosine_simil cosine_simil_neighbors \
isupper isupper_neighbors istitle istitle_neighbors'

options_string = 'left_neighbors=3 right_neighbors=3 one_hot one_hot_neighbors \
inside_paren pos chunk iob named_entity \
inside_paren_neighbors pos_neighbors chunk_neighbors iob_neighbors named_entity_neighbors \
chunk_end chunk_end_neighbors same_chunk_neighbors \
w2v_model=pubmed_wiki w2v w2v_neighbors w2v_size=30 \
cosine_simil cosine_simil_neighbors \
isupper isupper_neighbors istitle istitle_neighbors'

w2v=pubmed_wiki_w2v

features_name = 'P8'

In [ ]:
# Compute features for train
train_features = abstracts2features(train_tokens, train_genia_tags, w2v=w2v, options_string=options_string)

In [ ]:
# Compute features for dev
dev_features = abstracts2features(dev_tokens, dev_genia_tags, w2v=w2v, options_string=options_string)

In [ ]:
# Compute features for test
test_features = abstracts2features(test_tokens, test_genia_tags, w2v=w2v, options_string=options_string)

In [ ]:
# For debug
sanity_check(train_features)

Grid search

In [ ]:
# Run grid search
grid_file_name = 'crf_results/{}_grid'.format(features_name)
num_iters = 200
l1_list = [0.01, 0.1, 1]
l2_list = [0.01, 0.1, 1]

start_time = time.time()
grid_search_result = grid_search(train_features, train_tags, dev_features, dev_tags,\
                                 num_iters, l1_list, l2_list, eval_tags, file_name=grid_file_name, save=True)
print("--- %s seconds ---" % (time.time() - start_time))

os.remove(grid_file_name + '.model')

In [ ]:
# Sort result
sorted_result = sort_by_metric(grid_search_result, tag, metric='f1')
print_result(sorted_result)

Run CRF

In [ ]:
# Set options
num_iters = 200
l1 = 1
l2 = 0.01
file_name = 'crf_results/{}'.format(features_name)

In [ ]:
# Run CRF
start_time = time.time()
crf_result = get_crf_results(train_features, train_tags, dev_features, dev_tags, num_iters, l1, l2, eval_tags,
                             file_name=file_name, save=True)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Print result
print_result(crf_result)

Get model

In [ ]:
# Get model from file
tagger = get_tagger(file_name)

In [ ]:
# For debug
print_model_info(tagger)

Predict tags

In [ ]:
# Predict dev tags
pred_dev_tags = predict_tags(tagger, dev_features)

# Evaluate dev tags
dev_result = evaluate_prediction(pred_dev_tags, dev_tags, eval_tags)
print 'dev:'
print_result(dev_result)
sys.stdout.flush()

# Predict train tags
pred_train_tags = predict_tags(tagger, train_features)

# Evaluate train tags
train_result = evaluate_prediction(pred_train_tags, train_tags, eval_tags)
print 'train:'
print_result(train_result)
sys.stdout.flush()

# Predict test tags
pred_test_tags = predict_tags(tagger, test_features)

# Evaluate test tags
test_result = evaluate_prediction(pred_test_tags, test_tags, eval_tags)
print 'test:'
print_result(test_result)
sys.stdout.flush()

In [ ]:
write_result(pred_train_tags, file_name + '_pred_train_tags')
write_result(pred_dev_tags, file_name + '_pred_dev_tags')
write_result(pred_test_tags, file_name + '_pred_test_tags')

K-fold evaluation

In [ ]:
# Run K-fold
kfold_file_name = 'crf_results/{}_kfold'.format(features_name)

start_time = time.time()
kfold_result = get_kfold_results(train_features, train_tags, num_iters, l1, l2, eval_tags,\
                                 file_name=kfold_file_name, save=True)
print("--- %s seconds ---" % (time.time() - start_time))

os.remove(kfold_file_name + '.model')

In [ ]:
# Print all results
print_result(kfold_result)

Print a sample prediction for an abstract

In [ ]:
def print_with_spaces(l, spaces):
    # This pads strings to be of space length and aligned left
    formatter = lambda space: '{:' + str(space) + '}'
    
    for sublist in l:
        print ''.join([formatter(space).format(string) for string, space in zip(sublist, spaces)])

In [ ]:
i = 300
print_with_spaces(zip(dev_tokens[i], dev_tags[i], pred_dev_tags[i]), [25, 5, 5])

Analyze intervals

In [ ]:
compare_tags(pred_dev_tags, dev_tags, tag)

Restrict evaluation to noun phrase

In [ ]:
# Evaluate dev tags
dev_result = evaluate_prediction(filter_phrase(pred_dev_tags, dev_genia_tags),\
                                 filter_phrase(dev_tags, dev_genia_tags),\
                                 eval_tags)
print 'dev:'
print_result(dev_result)
sys.stdout.flush()

# Evaluate train tags
train_result = evaluate_prediction(filter_phrase(pred_train_tags, train_genia_tags),\
                                   filter_phrase(train_tags, train_genia_tags),\
                                   eval_tags)
print 'train:'
print_result(train_result)
sys.stdout.flush()

# Evaluate test tags
test_result = evaluate_prediction(filter_phrase(pred_test_tags, test_genia_tags),\
                                  filter_phrase(test_tags, test_genia_tags),\
                                  eval_tags)
print 'test:'
print_result(test_result)
sys.stdout.flush()